In [3]:
import numpy as np
import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import pandas as pd

In [8]:
train_classes = pd.read_json('datasets/labels/train.json').drop(['Background', 'Crack', 'Efflorescence', 'Spallation_ExposedBars_CorrosionStain'], axis=1)
print(train_classes.head())

                              img  combination_id
0  image_0000001_crop_0000001.png               1
1  image_0000001_crop_0000002.png               1
2  image_0000001_crop_0000003.png               1
3  image_0000001_crop_0000004.png               1
4  image_0000001_crop_0000005.png               1


In [18]:
import tensorflow as tf
import pandas as pd
import os

# Load the labels
cols_to_drop = ['Background', 'Crack', 'Efflorescence', 'Spallation_ExposedBars_CorrosionStain']
train_classes = pd.read_json(r'datasets\labels\train.json').drop(cols_to_drop, axis=1)
test_classes = pd.read_json(r'datasets\labels\test.json').drop(cols_to_drop, axis=1)
val_classes = pd.read_json(r'datasets\labels\val.json').drop(cols_to_drop, axis=1)


# Define base directories
train_dir = os.path.join(os.getcwd(), 'datasets', 'images', 'train')
val_dir = os.path.join(os.getcwd(), 'datasets', 'images', 'val')
test_dir = os.path.join(os.getcwd(), 'datasets', 'images', 'test')

# Helper function to create file paths and labels
def create_file_label_pairs(directory, label_df):
    file_paths = []
    labels = []

    for _, row in label_df.iterrows():
        image_path = os.path.join(directory, row['img'])
        if os.path.exists(image_path):
            file_paths.append(image_path)
            labels.append(row['combination_id'])
        else:
            print(f"Warning: File {image_path} not found and will be skipped.")

    return file_paths, labels

# Generate file paths and labels for each dataset
train_file_paths, train_labels = create_file_label_pairs(train_dir, train_classes)
val_file_paths, val_labels = create_file_label_pairs(val_dir, val_classes)
test_file_paths, test_labels = create_file_label_pairs(test_dir, test_classes)

# Create a TF dataset
def create_dataset(file_paths, labels, batch_size=32):
    # Create a dataset from file paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((file_paths, labels))

    # Function to load and preprocess images
    def load_and_preprocess(file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_png(img, channels=3)
        img = tf.image.resize(img, [224, 224])  # Resize to the input size
        img = img / 255.0  # Normalize to [0, 1]
        return img, label

    # Map preprocessing function and batch the dataset
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size=len(file_paths)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create datasets
train_dataset = create_dataset(train_file_paths, train_labels)
val_dataset = create_dataset(val_file_paths, val_labels)
test_dataset = create_dataset(test_file_paths, test_labels)

# Example: Iterate through the training dataset
for images, labels in train_dataset.take(1):
    print("Image batch shape:", images.shape)
    print("Label batch shape:", labels.shape)


Image batch shape: (32, 224, 224, 3)
Label batch shape: (32,)


In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')  # Adjust output for your classes
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(train_dataset, validation_data=val_dataset, epochs=10)

C:\Program Files\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 197s 652ms/step - accuracy: 0.3707 - loss: 3.8721 - val_accuracy: 0.4648 - val_loss: 1.4827
Epoch 2/10
287/287 ━━━━━━━━━━━━━━━━━━━━ 199s 652ms/step - accuracy: 0.5212 - loss: 1.2478 - val_accuracy: 0.4877 - val_loss: 1.4644
Epoch 3/10
239/287 ━━━━━━━━━━━━━━━━━━━━ 29s 610ms/step - accuracy: 0.5813 - loss: 1.0630

KeyboardInterrupt: 